In [3]:
import pandas as pd
from datetime import datetime, date

#Assign filenames
fullfile = 'Data/NM_2016_17export.csv'
ndfile = 'Data/2017 PMP Data Final No Duplicates.xlsx'

#import dataframes
fulldf = pd.read_csv(fullfile)
ND17df = pd.read_excel(ndfile)

#Convert the incident date column into a datetime object for both date frames
fulldf['INCIDENT DATE'] = pd.to_datetime(fulldf['INCIDENT DATE'])

datelist = ND17df['INCIDENT DATE'].tolist() # convert excel time delta into datetimes
datelist = [pd.to_datetime('1899-12-30') + pd.to_timedelta(int(date),unit='D') for date in datelist]
ND17df['INCIDENT DATE'] = datelist

#let's round of the times
fulldf['INCIDENT DATE'] = fulldf['INCIDENT DATE'].dt.normalize()
ND17df['INCIDENT DATE'] = ND17df['INCIDENT DATE'].dt.normalize()

#index dataframes by incident number - apparently could have done this within read.csv function
def incident_index(df):
    df.index = df['#']
    del df['#']
    df.index.name = "INCIDENT NUMBER"
    return df

fulldf = incident_index(fulldf)
ND17df = incident_index(ND17df)

#Sort the dfs by index
fulldf = fulldf.sort_index(ascending = False)
ND17df = ND17df.sort_index(ascending = False)

#Eyeball the difference between the No duplicates file and the strait export. 
fulldf.head() 
#ND17df.head()

,INCIDENT TITLE,INCIDENT DATE,LOCATION,DESCRIPTION,CATEGORY,LATITUDE,LONGITUDE,HLCIT CODE,LOCATION ACCURACY,Country,...,Total Vehicles,Severe Vehicle Damage,Election-related specific cause primary,Election-related specific cause secondary,Verification Feedback,FIRST NAME,LAST NAME,EMAIL,APPROVED,VERIFIED
INCIDENT NUMBER,,,,,,,,,,,,,,,,,,,,,
17685,Newspaper: Bank section of transportation mana...,2017-12-30,"Janakpur, Dhanusa, JANAKPUR, Central Developme...","Alert: Dhanusha/Dec 30, Yatayat byawastha kary...","News Media, Incident, Announcements / Demands,...",26.730245,85.927913,524 2 04 17 3 001,Municipality/VDC,Nepal,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,YES,NO
17684,Janakpur Express: Madhesh movement victims pro...,2017-12-30,"Janakpur, Dhanusa, JANAKPUR, Central Developme...","Alert: Dhanusha/Dec 30, Tesro madhesh andolan ...","LOCAL COMMUNITIES, Protest / Demonstration, Ne...",26.727996,85.927458,524 2 04 17 3 001,Municipality/VDC,Nepal,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,YES,NO
17675,Janakpur Express: Nawarang hotel vandalized | ...,2017-12-27,"Janakpur, Dhanusa, JANAKPUR, Central Developme...","Alert: Dhanusha/Dec 27, Janakpur ko shiva chow...","Nepal Police (responding), News Media, Vandali...",26.735339,85.936133,524 2 04 17 3 001,Municipality/VDC,Nepal,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,YES,NO
17670,Janakpur Today: Students padlocked technical s...,2017-12-23,"Dhanusadham, Dhanusa, JANAKPUR, Central Develo...","Alert: Dhanusha/Dec 23, Niyamit pathanpathan k...","News Media, District Education Office / MoE, I...",26.819670,86.055004,524 2 04 17 5 050,Municipality/VDC,Nepal,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,YES,NO
17668,Janakpur Today: Students protest in front of D...,2017-12-22,"Rajbiraj, Saptari, SAGARMATHA, Eastern Develop...","Alert: Saptari/Dec 22, Little angels boarding ...","Nepal Police (responding), Protest / Demonstra...",26.501006,86.772946,524 1 03 15 5 033,Municipality/VDC,Nepal,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,YES,NO


In [4]:
ND17df.head()

,INCIDENT TITLE,INCIDENT DATE,LOCATION,DESCRIPTION,CATEGORY,LATITUDE,LONGITUDE,HLCIT CODE,LOCATION ACCURACY,Sushila Karki,...,Youth -25 years or below- abducted,Total Buildings,Severe Building Damage,Total Vehicles,Severe Vehicle Damage,Feedback,Election-related specific cause primary,QC,Election-related specific cause secondary,Verification Feedback
INCIDENT NUMBER,,,,,,,,,,,,,,,,,,,,,
17687,Janakpur Today: RJPN cadres demonstrated again...,2017-12-31,"Gaur, Rautahat, NARAYANI, Central Development ...","Alert: Dhanusha/Dec 31, Samsad Anil Jha biurud...","Protest / Demonstration, News Media, Other Ele...",26.774839,85.258531,524 2 06 32 3 001,Municipality/VDC,NaN,...,0.0,0,0,0,0,NaN,Intraparty competition,Checked by Field,NaN,NaN
17685,Newspaper: Bank section of transportation mana...,2017-12-30,"Janakpur, Dhanusa, JANAKPUR, Central Developme...","Alert: Dhanusha/Dec 30, Yatayat byawastha kary...","News Media, Incident, Announcements / Demands,...",26.730245,85.927913,524 2 04 17 3 001,Municipality/VDC,NaN,...,0.0,0,0,0,0,NaN,NaN,Checked by Field,NaN,NaN
17684,Janakpur Express: Madhesh movement victims pro...,2017-12-30,"Janakpur, Dhanusa, JANAKPUR, Central Developme...","Alert: Dhanusha/Dec 30, Tesro madhesh andolan ...","LOCAL COMMUNITIES, Protest / Demonstration, Ne...",26.727996,85.927458,524 2 04 17 3 001,Municipality/VDC,NaN,...,0.0,0,0,0,0,NaN,NaN,Checked by Field,NaN,NaN
17675,Janakpur Express: Nawarang hotel vandalized | ...,2017-12-27,"Janakpur, Dhanusa, JANAKPUR, Central Developme...","Alert: Dhanusha/Dec 27, Janakpur ko shiva chow...","Nepal Police (responding), News Media, Vandali...",26.735339,85.936133,524 2 04 17 3 001,Municipality/VDC,NaN,...,0.0,1,0,0,0,NaN,NaN,Checked by Field,NaN,NaN
17670,Janakpur Today: Students padlocked technical s...,2017-12-23,"Dhanusadham, Dhanusa, JANAKPUR, Central Develo...","Alert: Dhanusha/Dec 23, Niyamit pathanpathan k...","News Media, District Education Office / MoE, I...",26.819670,86.055004,524 2 04 17 5 050,Municipality/VDC,NaN,...,0.0,0,0,0,0,NaN,NaN,Checked by Field,NaN,NaN


In [5]:

#Ensure the dates match for any given item / find where they don't
#mask = fulldf.mask(fulldf['INCIDENT DATE'] != ND17df['INCIDENT DATE']) <-- won't work becasue index is of different length
def match_check(df1,df2,column_name):  #lets make it a function so we don't have to replicate it
    for num in df1.index:
        if (num in df2.index) and (df1.loc[num][column_name] != df2.loc[num][column_name]):
            return False
    return True

print('Dates Match =', match_check(fulldf,ND17df,'INCIDENT DATE'))



Dates Match = True


In [6]:
#compare 1 full df to duplicate df to see what has been removed
def makediff(df1,df2):
    df1['difflist'] = [num not in df2.index for num in df1.index] #creat a column of bools in df1-true if they are not in df2's index
    diffdf = df1.loc[df1['difflist'] == True] # Diffdf is now a slice of what is in df1 but not df2
    del diffdf['difflist'] # remove the new column
    return diffdf

#Slice fulldf down to only 2017
full17df = fulldf.loc[fulldf['INCIDENT DATE'].dt.year == 2017]

#calculate difference
diffdropped = makediff(full17df, ND17df)  # These items were removed from the 2017 data to make the no duplicates file '
diff17df = makediff(ND17df, full17df)  # This should what was added into the "No duplicates" file 'ND17df'

statsdf = pd.DataFrame(index=['full17df','ND17df','diff17df','diffdropped'], columns = ['shape','comments'])
statsdf['comments'] = ['The original 2017 export','No duplicates file','Added to make no duplicates file','The duplicates - Removed to make ND17df']
statsdf.index.name = 'Dataframe'
statsdf['shape'] = [full17df.shape, ND17df.shape, diff17df.shape, diffdropped.shape]
statsdf.head()

/home/nhorning/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,shape,comments
Dataframe,,
full17df,"(4115, 77)",The original 2017 export
ND17df,"(3555, 70)",No duplicates file
diff17df,"(13, 69)",Added to make no duplicates file
diffdropped,"(573, 76)",The duplicates - Removed to make ND17df


In [7]:
diff17df #These were what was added to make ND17df. They are all on the last day, besides the two from 2018. Probably should be in final file

,INCIDENT TITLE,INCIDENT DATE,LOCATION,DESCRIPTION,CATEGORY,LATITUDE,LONGITUDE,HLCIT CODE,LOCATION ACCURACY,Sushila Karki,...,Youth -25 years or below- abducted,Total Buildings,Severe Building Damage,Total Vehicles,Severe Vehicle Damage,Feedback,Election-related specific cause primary,QC,Election-related specific cause secondary,Verification Feedback
INCIDENT NUMBER,,,,,,,,,,,,,,,,,,,,,
17687,Janakpur Today: RJPN cadres demonstrated again...,2017-12-31,"Gaur, Rautahat, NARAYANI, Central Development ...","Alert: Dhanusha/Dec 31, Samsad Anil Jha biurud...","Protest / Demonstration, News Media, Other Ele...",26.774839,85.258531,524 2 06 32 3 001,Municipality/VDC,NaN,...,0.0,0,0,0,0,NaN,Intraparty competition,Checked by Field,NaN,NaN
17506,INSEC: Arrested on charge of child molestation...,2018-01-01,"Rajbiraj-10, Saptari, SAGARMATHA, Eastern Deve...","Alert: Saptari/Jan 2, 12 barshiya balika lai y...","Trusted Reports, INSEC, Nepal Police (respondi...",26.537218,86.750468,524 1 03 15 3 001 3,Ward,NaN,...,0.0,0,0,0,0,NaN,NaN,Checked by Admin,NaN,NaN
17495,eKantipur: Two workers found dead | à¤®à¥‹à¤¬à...,2017-12-31,"Pokhara Lekhnath, Kaski, GANDAKI, Western Deve...","Alert: Kaski/Jan 1, Mobile chore pachhi gaali-...","Killings, INJURED, DEATHS, Nepal Police (respo...",28.243930,83.948625,524 3 07 39 5 043,Municipality/VDC,NaN,...,0.0,0,0,0,0,NaN,NaN,Checked by Admin,NaN,NaN
17492,Annapurna: Locals of Dipayal protest demanding...,2017-12-31,"Dipayal Silgadi, Doti, SETI, Far-Western Devel...","Alert: Doti/Jan 1, Pradesh 7 ko rajdhani Dipay...","LOCAL COMMUNITIES, Protest / Demonstration, Ne...",29.282945,80.891709,524 5 13 70 3 001,Municipality/VDC,NaN,...,0.0,0,0,0,0,NaN,NaN,Checked by Field,NaN,NaN
17486,News: Bandh called protesting change of rural ...,2017-12-31,"Suwarnabati, Rolpa, RAPTI, Mid-Western Develop...","Alert: Rolpa/Dec 31, Suvarnawoti gaunpalika ko...","LOCAL COMMUNITIES, News Media, Strike, Inciden...",28.198278,82.761318,524 4 10 54 5 005,Municipality/VDC,NaN,...,0.0,0,0,0,0,NaN,NaN,Checked by Admin,NaN,NaN
17484,"Alert: Man found dead in Banglachuli, suspect ...",2017-12-31,"Ghorahi-6, Dang, RAPTI, Mid-Western Developmen...","Alert: Dang/Jan 1, Bangalachuli gaunpalika maa...","Killings, DEATHS, Nepal Police (responding), D...",27.996877,82.449305,524 4 10 56 5 021 2,Ward,NaN,...,0.0,0,0,0,0,NaN,NaN,Check,NaN,NaN
17482,News: Pressure rally organized demanding Birat...,2018-01-01,"Biratnagar, Morang, KOSHI, Eastern Development...","Alert: Parsa/Jan 1, Biratnagar lai pradesh 1 k...","Protest / Demonstration, News Media, Incident,",26.484876,87.289331,524 1 02 05 2 001,Municipality/VDC,NaN,...,0.0,0,0,0,0,NaN,NaN,Checked by Admin,NaN,NaN
17479,News: Protest continues in Dhankuta | à¤§à¤¨à¤...,2017-12-31,"Dhankuta, Dhankuta, KOSHI, Eastern Development...","Alert: Dhankuta/Dec 31, Dhankuta lai pradesh 1...","Protest / Demonstration, News Media, Incident,",27.032196,87.318782,524 1 02 07 3 001,Municipality/VDC,NaN,...,0.0,0,0,0,0,0,NaN,Checked by Admin,NaN,NaN
17477,News: Woman kills husband by hitting with a wo...,2017-12-31,"Bhimsen-8, Gorkha, GANDAKI, Western Developmen...","Alert: Gorkha/Dec 31, Gharelu jhagada ko kram ...","Killings, DEATHS, Trusted Reports, Domestic Vi...",27.982000,84.751000,524 3 07 36 5 004 5,Ward,NaN,...,0.0,0,0,0,0,NaN,NaN,Checked by Admin,NaN,NaN


In [8]:
diffdropped.head()

,INCIDENT TITLE,INCIDENT DATE,LOCATION,DESCRIPTION,CATEGORY,LATITUDE,LONGITUDE,HLCIT CODE,LOCATION ACCURACY,Country,...,Total Vehicles,Severe Vehicle Damage,Election-related specific cause primary,Election-related specific cause secondary,Verification Feedback,FIRST NAME,LAST NAME,EMAIL,APPROVED,VERIFIED
INCIDENT NUMBER,,,,,,,,,,,,,,,,,,,,,
17599,Sajha kura: A man arrested with homemade gun |...,2017-12-28,"Makwanpur, NARAYANI, Central Development Regio...","Alert: Makwanpur/Dec 8, Bharuwa banduk sahit G...","Small Arms, Nepal Police (responding), ARREST,...",27.407938,85.117914,524 2 06 31,District,Nepal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,NO
17593,News:Engineer yadav arrested | इन्जिनीर यादव ...,2017-12-18,"Lahan, Siraha",Alert:Siraha/Dec 17; Siraha ka ek jana enginee...,"ARREST, Corruption / Fraud, News Media, CIAA, ...",26.713281,86.470451,524 1 03 16 3 002,Municipality/VDC,Nepal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,NO
17591,News: Woman rescued from being trafficked | बे...,2017-12-18,"Mechinagar, Jhapa, MECHI, Eastern Development ...","Alert: Parsa/Dec 18, Ramro rojgari ko lagi Del...","Human Trafficking, Nepal Police (responding), ...",26.770943,88.168507,524 1 01 04 5 018,Municipality/VDC,Nepal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,NO
17582,News: Parent mistreated by a teacher | शिक्षकद...,2017-12-14,"Mahagadhimai, Bara, NARAYANI, Central Developm...","Alert: Bara/Dec 15, Mahagadhimai nagarpalika-1...","Threat, News Media, Incident,",27.021855,85.059926,524 2 06 33 5 029,Municipality/VDC,Nepal,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,YES,NO
17550,Janakpur Today: A man arrested with bullets | ...,2017-12-24,"Gaur-2, Rautahat, NARAYANI, Central Developmen...","Alert: Rautahat/Dec 26, 1 than goli ra pechkas...","Small Arms, Nepal Police (responding), ARREST,...",26.761275,85.266560,524 2 06 32 3 001 5,Ward,Nepal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,NO


In [9]:
#lets see if we can replicate diffdropped with our own function. Duplcates can by flagged
#by finding any records with both matching date and place. HLCIT codes store places in a hierarchy.

#maybe we can count duplicate place codes after grouping by date. 
dupsgroup = full17df.groupby(['INCIDENT DATE','HLCIT CODE']).count()

#create a boolean array of HLCIT codes in which their are more than one incident on a given incident date
dupsflag = dupsgroup['INCIDENT TITLE'] > 1 #we can use incident title here because we used count and not count unqique
dupsflag.name = 'DUPLICATE FLAG' # make dups flag into a multi-indexed boolean

#do a join with the original list on the dates and codes. 
dupsjoin = full17df.join(dupsflag, on=('INCIDENT DATE','HLCIT CODE'), how='left') 

dupsdf = dupsjoin.loc[dupsjoin['DUPLICATE FLAG'] == True] 

#Drop rows where HLCIT code is too short to be very local. 
for num in dupsdf.index:
    if len(dupsdf.loc[num]['HLCIT CODE']) <= 11:
        dupsdf = dupsdf.drop(num)

print('Dupsdf shape: ',dupsdf.shape)
dupsdf.head()


Dupsdf shape:  (249, 78)


,INCIDENT TITLE,INCIDENT DATE,LOCATION,DESCRIPTION,CATEGORY,LATITUDE,LONGITUDE,HLCIT CODE,LOCATION ACCURACY,Country,...,Election-related specific cause primary,Election-related specific cause secondary,Verification Feedback,FIRST NAME,LAST NAME,EMAIL,APPROVED,VERIFIED,difflist,DUPLICATE FLAG
INCIDENT NUMBER,,,,,,,,,,,,,,,,,,,,,
17685,Newspaper: Bank section of transportation mana...,2017-12-30,"Janakpur, Dhanusa, JANAKPUR, Central Developme...","Alert: Dhanusha/Dec 30, Yatayat byawastha kary...","News Media, Incident, Announcements / Demands,...",26.730245,85.927913,524 2 04 17 3 001,Municipality/VDC,Nepal,...,NaN,NaN,NaN,NaN,NaN,NaN,YES,NO,False,True
17684,Janakpur Express: Madhesh movement victims pro...,2017-12-30,"Janakpur, Dhanusa, JANAKPUR, Central Developme...","Alert: Dhanusha/Dec 30, Tesro madhesh andolan ...","LOCAL COMMUNITIES, Protest / Demonstration, Ne...",26.727996,85.927458,524 2 04 17 3 001,Municipality/VDC,Nepal,...,NaN,NaN,NaN,NaN,NaN,NaN,YES,NO,False,True
17600,Dhangadhi Post: FSU and employees padlocked Da...,2017-12-21,"Amargadhi, Dadeldhura, MAHAKALI, Far-Western D...","Alert: Dadeldhura/Dec 21, Campus pramukh ko ra...","STUDENT UNIONS AFFILIATED TO POLITICAL PARTY, ...",29.302073,80.593389,524 5 14 73 3 001,Municipality/VDC,Nepal,...,NaN,NaN,NaN,NaN,NaN,NaN,YES,NO,False,True
17472,NayaPatrika: Maoist Center leaders misbehaved ...,2017-12-29,"Kathmandu, Kathmandu, BAGMATI, Central Develop...","Alert: Kathmandu/Dec 30, Samanupatik tarfa pra...","CPN (Maoist Center), Assault / Violent Acts, N...",27.713335,85.323148,524 2 05 27 1 001,Municipality/VDC,Nepal,...,Intraparty competition,NaN,NaN,NaN,NaN,NaN,YES,NO,False,True
17462,News: NSU takes to streets demanding relocatio...,2017-12-29,"Kathmandu, Kathmandu, BAGMATI, Central Develop...","Alert: Kathmandu/Dec 29, Gyaas udhyog basti ba...","News Media, Incident, Announcements / Demands,",27.713335,85.323148,524 2 05 27 1 001,Municipality/VDC,Nepal,...,NaN,NaN,NaN,NaN,NaN,NaN,YES,NO,False,True


In [10]:
#There are apparently 249 incidents that happened on the same date in the same locale 
#Let's view them. 
def print_front(df): #with a function so we don't have to copy paste
    df = df.sort_values(by='INCIDENT DATE')
    print(df.loc[:][['HLCIT CODE','INCIDENT DATE','INCIDENT TITLE']])

print_front(dupsdf) #obviously some are duplicates, see incident 17437. 

                           HLCIT CODE INCIDENT DATE  \
INCIDENT NUMBER                                       
12819               524 1 03 14 3 001    2017-01-04   
12822               524 1 03 14 3 001    2017-01-04   
13443               524 5 14 72 3 001    2017-01-16   
13447               524 5 14 72 3 001    2017-01-16   
13081               524 4 11 57 3 001    2017-01-22   
13014               524 4 11 57 3 001    2017-01-22   
13018               524 2 05 27 1 001    2017-01-23   
13020               524 2 05 27 1 001    2017-01-23   
13042               524 2 05 27 1 001    2017-01-25   
13064               524 2 05 27 1 001    2017-01-25   
13077               524 1 02 05 2 001    2017-01-25   
13127               524 2 04 17 3 001    2017-01-25   
13131               524 2 04 17 3 001    2017-01-25   
13439               524 1 02 05 2 001    2017-01-25   
13070               524 2 06 33 3 001    2017-01-26   
13084               524 2 06 33 3 001    2017-01-26   
13202     

In [11]:

#Compare shapes of diffs of dropped to this data set.
print('diffdropped:', diffdropped.shape)
imissed = makediff(diffdropped, dupsdf) #This shoud be the dulicates I missed
print('imissed:', imissed.shape)
theymissed = makediff(dupsdf, diffdropped)#this should be what they didn't remove but are next to each other on the same date
print('theymissed:', theymissed.shape)

#doesn't look like they intersect very much, maybe save as a file and go over manually, print side by side? 

diffdropped: (573, 76)
imissed: (549, 76)
theymissed: (225, 77)


/home/nhorning/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [105]:
# lets make them a multi index again to easily check for duplicates happening on the same date
def find_duplicate(df,column_name):
    multidups = df
    multidups['INCIDENT NUMBER'] = multidups.index.tolist()
    multidups = multidups.set_index(['INCIDENT DATE','HLCIT CODE','INCIDENT NUMBER'])
    multidups = multidups.sort_index()
   
    #check for duplicates in column blocks by date
    dup_count = 0
    dup_dict = {}
    for date in set(df['INCIDENT DATE']):  
        columnblock = multidups.loc[date][column_name]
        print_items = []
        for item in columnblock:
            if type(item) is str and columnblock.tolist().count(item) > 1:
                print_items.append(item)
                dup_count += 1
        print_items = list(set(print_items))
        if print_items != []:
            item_list = []
            for print_item in print_items:
                print('Duplicate', column_name,'found for:',date,'\n',columnblock[columnblock == print_item],'\n')
                #dup_dict[date] = columnblock.index[columnblock == print_item].tolist()
                item_list.append(columnblock.index[columnblock == print_item].tolist())
            #print(item_list) #debug
            dup_dict[date] = item_list
    print('Total Duplicate',column_name, 'found:', dup_count,'\n')
    dup_discriptions(dup_dict, df)
    return dup_dict
    #Also return a dictionary of duplicate incident mumbers, for printing. 

def dup_discriptions(dup_dict,df):
    for date in dup_dict.keys():
        print('Possible duplicates for:', date)
        for item in dup_dict[date]:
            for item in item:
                code = item[0]
                num = item[1]   
                print('TITLE:', df.loc[num]['INCIDENT TITLE'])
                print('lat/lon:', df.loc[num]['LATITUDE'], df.loc[num]['LONGITUDE'])
                print('HLCIT CODE:', code) 
                print(df.loc[num][['District','Municipality/VDC','Ward','Municipality/VDC(New)','Ward(New)']],'\n')
                print('Description', df.loc[num]['DESCRIPTION'],'\n')


#Check for duplicate titles on the same date in other data sets 
dup_dict_test = find_duplicate(dupsdf,'INCIDENT TITLE')
diffdroppeddt = find_duplicate(diffdropped,'INCIDENT TITLE')


#We can see that two duplicate titles were found on the same date in the same place in the Full 2017 export. 
#However, in the file of duplicates that were already dropped we can see two identical titles on the same date, 
#but a different location. This could means we cant count on duplcates to always be listed in the same location.

#However, the discription of the tuplicate titles in different locations shows, they are completely different incidents,
#while the duplicates titles found in the same location have exactly the same description. Let's make a note to delete them.

Duplicate INCIDENT TITLE found for: 2017-12-22 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 1 02 06 5 007  17436              News: E-rickshaw operators and police clash in...
                   17437              News: E-rickshaw operators and police clash in...
Name: INCIDENT TITLE, dtype: object 

Total Duplicate INCIDENT TITLE found: 2 

Possible duplicates for: 2017-12-22 00:00:00
TITLE: News: E-rickshaw operators and police clash in Itahari | रिक्सा चालक र प्रहरीबीच झडप ।
lat/lon: 26.6882 87.263291
HLCIT CODE: 524 1 02 06 5 007
District                    Sunsari
Municipality/VDC         Hanshposha
Ward                            NaN
Municipality/VDC(New)       Itahari
Ward(New)                       NaN
Name: 17436, dtype: object 

Description Alert: Sunsari/Dec 22, Rajmarga ma City Safari gudauna pratibandha lagayeko birodh ma utriyeka chalak ra prahari bich jhadap. 

A clash erupted between e-rickshaw operators and police in Itahari Municipality. The clash took place when

/home/nhorning/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


## DROP incident no. 17437 out of final data
#Lets feed the other datasets through. To see what else we can find.


In [106]:
print(statsdf,'\n')
print(statsdf.loc['full17df'].to_frame().T,'|----------------------------------\n')
full17dupsbytitle = find_duplicate(full17df,'INCIDENT TITLE')

                  shape                                 comments
Dataframe                                                       
full17df     (4115, 77)                 The original 2017 export
ND17df       (3555, 70)                       No duplicates file
diff17df       (13, 69)         Added to make no duplicates file
diffdropped   (573, 76)  The duplicates - Removed to make ND17df 

               shape                  comments
full17df  (4115, 77)  The original 2017 export |----------------------------------

Duplicate INCIDENT TITLE found for: 2017-05-02 00:00:00 
 HLCIT CODE            INCIDENT NUMBER
524 3 09 48 3 002 11  14346              INSEC: One arrested on charge of rape case | ब...
524 3 09 48 5 013 3   14348              INSEC: One arrested on charge of rape case | ब...
Name: INCIDENT TITLE, dtype: object 

Duplicate INCIDENT TITLE found for: 2017-06-04 00:00:00 
 HLCIT CODE           INCIDENT NUMBER
524 4 11 57 5 012    14901              INSEC: Accused rapist arre

/home/nhorning/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.



 HLCIT CODE           INCIDENT NUMBER
524 1 02 05 5 051    14949              INSEC: Rape accused arrested | बलात्कार घटनाका...
524 2 06 31 5 008 4  14961              INSEC: Rape accused arrested | बलात्कार घटनाका...
524 3 07 36 5 016 2  14966              INSEC: Rape accused arrested | बलात्कार घटनाका...
Name: INCIDENT TITLE, dtype: object 

Duplicate INCIDENT TITLE found for: 2017-01-13 00:00:00 
 HLCIT CODE           INCIDENT NUMBER
524 1 01 02 5 020 1  12939              INSEC: Accused bigamous arrested | बहुविवाह घट...
524 1 03 15 5 049 3  12942              INSEC: Accused bigamous arrested | बहुविवाह घट...
Name: INCIDENT TITLE, dtype: object 

Duplicate INCIDENT TITLE found for: 2017-10-07 00:00:00 
 HLCIT CODE           INCIDENT NUMBER
524 3 09 51 5 079 6  16359              INSEC: Rape accused arrested | बलात्कार घटनाका...
524 5 13 68 5 013 8  16364              INSEC: Rape accused arrested | बलात्कार घटनाका...
Name: INCIDENT TITLE, dtype: object 

Duplicate INCIDENT TITLE fo

Possible duplicates for: 2017-12-22 00:00:00
TITLE: News: E-rickshaw operators and police clash in Itahari | रिक्सा चालक र प्रहरीबीच झडप ।
lat/lon: 26.6882 87.263291
HLCIT CODE: 524 1 02 06 5 007
District                    Sunsari
Municipality/VDC         Hanshposha
Ward                            NaN
Municipality/VDC(New)       Itahari
Ward(New)                       NaN
Name: 17436, dtype: object 

Description Alert: Sunsari/Dec 22, Rajmarga ma City Safari gudauna pratibandha lagayeko birodh ma utriyeka chalak ra prahari bich jhadap. 

A clash erupted between e-rickshaw operators and police in Itahari Municipality. The clash took place when e-rickshaw operators staged protest against a new rule of banning three-wheelers.

राजमार्गमा गुडाउन प्रतिबन्ध लगाएको विरोधमा उत्रिएका विद्युतीय रिक्सा (सिटी सफारी) चालक र प्रहरीबीच झडप भएको छ। झडपमा ६ राउन्ड अ श्रुग्यास प्रहार गरेको इटहरी इलाका प्रहरी प्रमुख डीएसपी सञ्जय राउतले जानकारी दिए।

रिक्सा दर्ता गर्दा अनुमति पाएको रुटमा मात्र चलाउन दिने

## 1 Duplicate pair found for the original 2017 export (full17df)

A definate duplicate pair is shown on 2017-12-22 (17436, 17437), as the discription matches completely. 

Another pair on 17-12-06 (17251, 17253) could consists of two bombs in places apparently next to each other.
They could be investigated further. Nepal's transition to federalism has created a new naming scheme, listing the new wards next to their descriptions shows that they are indeed mapped in seperate appropriate places in the new naming scheme. Googling the gps coordinates shows they are about 30km apart, and can't be ruled out as seperate incidents.

Was it caught by the team who made the no duplicates file?


In [107]:
#print(full17df.columns)
print(statsdf.loc['ND17df'].to_frame().T,'|----------------------------------\n')
ND17dupdt = find_duplicate(ND17df,'INCIDENT TITLE')

             shape            comments
ND17df  (3555, 70)  No duplicates file |----------------------------------

Duplicate INCIDENT TITLE found for: 2017-05-02 00:00:00 
 HLCIT CODE            INCIDENT NUMBER
524 3 09 48 3 002 11  14346              INSEC: One arrested on charge of rape case | à...
524 3 09 48 5 013 3   14348              INSEC: One arrested on charge of rape case | à...
Name: INCIDENT TITLE, dtype: object 

Duplicate INCIDENT TITLE found for: 2017-06-04 00:00:00 
 HLCIT CODE           INCIDENT NUMBER
524 4 11 57 5 012    14901              INSEC: Accused rapist arrested | à¤¬à¤²à¤¾à¤¤à...
524 4 11 57 5 013 3  14902              INSEC: Accused rapist arrested | à¤¬à¤²à¤¾à¤¤à...
Name: INCIDENT TITLE, dtype: object 

Duplicate INCIDENT TITLE found for: 2017-06-08 00:00:00 
 HLCIT CODE           INCIDENT NUMBER
524 1 02 05 5 051    14949              INSEC: Rape accused arrested | à¤¬à¤²à¤¾à¤¤à¥...
524 2 06 31 5 008 4  14961              INSEC: Rape accused arrested | 

## No other duplicates found,  but the E-rickshaw operator clash on 2017-12-22 (17436,17437) is still present. 

Excel has garbled the nepali text. We'll just have to index one by the other when we get there.
Also remember to run the VDC columns through the duplicate function to get duplicates by location. Search for duplicates within a date range using a date delta?  
Idea!!!  check if any name is included in the other one! just check if num + years old patern matches

This means that most of the duplicates the team dropped had to be mostly from manual checking, and they missed an obvious one. Let's see if we get any more identified duplicates by finding duplicate locations by date. 

In [108]:
#check for duplicates by VDC in original 2017 export
full17dupsbyvdc = find_duplicate(full17df,'Municipality/VDC')

/home/nhorning/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Duplicate Municipality/VDC found for: 2017-06-02 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 4 11 57 3 001  15066              Nepalgunj N.P.
                   15074              Nepalgunj N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-01-21 00:00:00 
 HLCIT CODE           INCIDENT NUMBER
524 5 14 72 5 002    13012              Beldandi
524 5 14 72 5 002 2  13011              Beldandi
524 5 14 72 5 002 5  13010              Beldandi
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-03-13 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 1 02 05 2 001  13692              Biratnagar U.M.N.P.
                   13765              Biratnagar U.M.N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-03-22 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 1 02 05 2 001  14085              Biratnagar U.M.N.P.
                   14088              Biratnagar U.M.N.P.
Name: Municip

 HLCIT CODE         INCIDENT NUMBER
524 2 06 35 3 001  14371              Bharatpur N.P.
                   14763              Bharatpur N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-12-21 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 5 14 73 3 001  17394              Amargadhi N.P.
                   17600              Amargadhi N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-08-28 00:00:00 
 HLCIT CODE            INCIDENT NUMBER
524 2 05 27 1 001 13  15973              Kathmandu M.N.P.
524 2 05 27 1 001 3   15979              Kathmandu M.N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-03-30 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 2 04 17 3 001  13978              Janakpur N.P.
                   13979              Janakpur N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-09-25 00:00:00 
 HLCIT CODE            IN

Duplicate Municipality/VDC found for: 2017-03-28 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 2 04 17 3 001  13936              Janakpur N.P.
                   13952              Janakpur N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-03-28 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 2 06 35 3 001  13955              Bharatpur N.P.
                   13984              Bharatpur N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-06-18 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 3 09 47 3 001  15078              Ramgram N.P.
                   15111              Ramgram N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-03-02 00:00:00 
 HLCIT CODE            INCIDENT NUMBER
524 2 05 27 1 001 10  13487              Kathmandu M.N.P.
524 2 05 27 1 001 31  13495              Kathmandu M.N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC fo

Duplicate Municipality/VDC found for: 2017-07-13 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 1 02 05 2 001  15531              Biratnagar U.M.N.P.
                   15533              Biratnagar U.M.N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-07-07 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 2 04 17 3 001  15488              Janakpur N.P.
                   15490              Janakpur N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-09-06 00:00:00 
 HLCIT CODE            INCIDENT NUMBER
524 2 05 27 1 001 11  16093              Kathmandu M.N.P.
524 2 05 27 1 001 4   16079              Kathmandu M.N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-09-06 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 2 06 32 5 094  16085              Chandranigahapur
                   16097              Chandranigahapur
Name: Municipality/VDC, dtype: object 

Duplicate M

...

TITLE: News: 14-year-old boy held on charge of knife attack | चक्कु प्रहार गर्ने पक्राउ।
lat/lon: 27.712239 85.297544
HLCIT CODE: 524 2 05 27 1 001
District                        Kathmandu
Municipality/VDC         Kathmandu M.N.P.
Ward                                  NaN
Municipality/VDC(New)           Kathmandu
Ward(New)                             NaN
Name: 13631, dtype: object 

Description Alert: Kathmandu/Mar 10, 19 barshiya yuwak lai chakku prahar gareko abhiyog ma Bihibar prahari dwara Bijeswori basne 14 barshiya ek kishor pakrau.

Police on Thursday arrested a 14-year-old boy for his alleged involvement in attacking Dipendra Pariyar, 19, of Maidi-7, Dhading, currently residing at Bijeshwori, KMC-15, with a knife.

The suspect has been identified as Sangam Tamang of Chilime, Rasuwa, presently living at Bijeshwori, KMC-14.

Police said that Tamang allegedly attacked Pariyar with the knife over a minor dispute.

Injured in the incident, Pariyar was initially taken to the Manmoha

HLCIT CODE: 524 5 13 69 5 023 2
District                      Achham
Municipality/VDC         Khodasadevi
Ward                               2
Municipality/VDC(New)        Mellekh
Ward(New)                          6
Name: 15367, dtype: object 

Description Alert: Achham/Jun 28, Mellekh-6 Sodasha ma Emaale samarthit sikshyak ko kutpit bata ek janaa Nepali Congress samarthit bridhha ghaite.

A Nepali Congress supporter was injured after being beaten by a group led by CPN UML affiliated teacher in ShodaSha of Mellekh rural municipality-6 on Wednesday. Lachhiram Budha, 60, was beaten by the group while he was returning home after voting, informed Lachhiram's grandson Padam Budha. Lachhiram, who is also former village committee chairman of Nepali Congress sustained injuries on different parts of his body due to the beating. UML cadres Prithvi and Janak Rawal had beaten him over a dispute they had ran into with Lachhiram that afternoon. Police are believed to have headed for the site of the

TITLE: News: Police arrests man on the charge of knife attack on wife | श्रीमतीलाई छुरी हान्ने पक्राउ।
lat/lon: 28.120214 82.299171
HLCIT CODE: 524 4 10 56 3 001 6
District                          Dang
Municipality/VDC         Tulsipur N.P.
Ward                                 6
Municipality/VDC(New)         Tulsipur
Ward(New)                            6
Name: 15540, dtype: object 

Description Alert: Dang/Jul 13, Afnai shrimati lai chhura prahaar gari ghaite banaaeyko arop ma Tulsipur upa-mahanagarpalika ka ek 29 barshiya yuwak pakrau.

Area Police Office Tulsipur arrested 29-year-old Dipendra Gupta, a resident of Tulsipur sub-municipality-6 on the charge of attacking his wife, Shanti Gupta, with knife during a domestic dispute at noon on July 12. His wife is currently being treated in Rapti Zonal Hospital, Tulsipur. Police are further investigating on the incident.

प्रहरी कार्यालय तुलसीपुरले आफ्नै श्रीमतीलाई छुरा प्रहार गरि गम्भिर घाइते तुल्याउने एकजना व्यक्तिलाई तुलसीपुर बजारवाट 

Description Alert: Humla/May 14, Kharpunath gaunpalika-5 ma Congress ka 2 pakchhya bich ko jhadap rokna prahari dwara 2 round hawai fire.

Situation at a election center in Kharpunath Village Council-5 of Humla district became tense for a while when supporters of Nepali Congress official and rebel candidates clashed during local bodies election on Sunday.

Police had to fire two shots in the air to take the situation under control when supporters of NC's official candidate Bhim Bahadur Shahi and rebel candidate Prakash Bahadur Shahi pelted stones at each other during the voting.

Ministry for Water Supply &amp;amp; Sanitation Jivan Bahadur Shahi, who hails and is currently in Kharpu, summoned further security forces after the clash.

Voting had to be stopped for nearly 30 minutes during the scuffle. DSP Kamal Nepali said the situation has not returned to normalcy and voting has now resumed at the center.

 
हुम्लाको खार्पुनाथ गाँउपालिकाको वडा नम्बर ५ मा काग्रेस दुई समूहबीच तनाव हुँदा स

TITLE: Krishna: Locals demonstrated for halting construction of Hulaki highway in the city | नगर हुँदै हुलाकी राजमार्ग नलैजान भन्दै स्थानीयद्वारा प्रदर्शन  ।
lat/lon: 26.541946 86.756722
HLCIT CODE: 524 1 03 15 3 001
District                       Saptari
Municipality/VDC         Rajbiraj N.P.
Ward                               NaN
Municipality/VDC(New)         Rajbiraj
Ward(New)                          NaN
Name: 13717, dtype: object 

Description Alert: Saptari/Mar 1, nagar hudai hulaki rajmarg nirmaan garda sthaniye ghar pariwar uthibas hune vandai sthaniye dwara rajbiraj ma pradarshan. 

locals have staged a protest demonstration in Rajbiraj against the proposed construction of Hulaki highway along the municipality on Tuesday. They said that the construction would cause great loss of property leaving people to homeless. Staging the demonstrators they handed over the memorandum to Prime Minister through CDO and condemned organizing a press meeting. In the memorandum, they suggested 

In [109]:
#Check in no duplicate file for dups
ND17dupsdfbyvdc = find_duplicate(ND17df,'Municipality/VDC')

Duplicate Municipality/VDC found for: 2017-06-02 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 4 11 57 3 001  15066              Nepalgunj N.P.
                   15074              Nepalgunj N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-01-21 00:00:00 
 HLCIT CODE           INCIDENT NUMBER
524 5 14 72 5 002    13012              Beldandi
524 5 14 72 5 002 2  13011              Beldandi
524 5 14 72 5 002 5  13010              Beldandi
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-03-13 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 1 02 05 2 001  13692              Biratnagar U.M.N.P.
                   13765              Biratnagar U.M.N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-03-22 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 1 02 05 2 001  14085              Biratnagar U.M.N.P.
                   14088              Biratnagar U.M.N.P.
Name: Municip

Duplicate Municipality/VDC found for: 2017-02-12 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 1 03 12 5 044  13712              Diktel
                   14124              Diktel
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-03-10 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 2 05 27 1 001  13631              Kathmandu M.N.P.
                   13632              Kathmandu M.N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-01-23 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 2 05 27 1 001  13018              Kathmandu M.N.P.
                   13020              Kathmandu M.N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-04-16 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 2 06 34 2 001  14175              Birgunj U.M.N.P.
                   14184              Birgunj U.M.N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 20

Duplicate Municipality/VDC found for: 2017-10-09 00:00:00 
 HLCIT CODE            INCIDENT NUMBER
524 2 05 27 1 001     16395              Kathmandu M.N.P.
524 2 05 27 1 001 1   16396              Kathmandu M.N.P.
524 2 05 27 1 001 31  16399              Kathmandu M.N.P.
524 2 05 27 1 001 34  16378              Kathmandu M.N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-11-26 00:00:00 
 HLCIT CODE           INCIDENT NUMBER
524 5 14 75 5 012 2  17023              Khalanga
524 5 14 75 5 012 5  17050              Khalanga
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-11-26 00:00:00 
 HLCIT CODE            INCIDENT NUMBER
524 2 04 22 3 001     17029              Bhimeswor N.P.
                      17030              Bhimeswor N.P.
524 2 04 22 3 001 1   17043              Bhimeswor N.P.
524 2 04 22 3 001 10  17044              Bhimeswor N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found 

Duplicate Municipality/VDC found for: 2017-07-16 00:00:00 
 HLCIT CODE           INCIDENT NUMBER
524 1 02 06 3 002    15557              Itahari N.P.
524 1 02 06 3 002 5  15625              Itahari N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-06-21 00:00:00 
 HLCIT CODE           INCIDENT NUMBER
524 1 02 06 3 002    15419              Itahari N.P.
524 1 02 06 3 002 8  15143              Itahari N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-03-03 00:00:00 
 HLCIT CODE         INCIDENT NUMBER
524 1 03 15 3 001  13502              Rajbiraj N.P.
                   13711              Rajbiraj N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipality/VDC found for: 2017-08-29 00:00:00 
 HLCIT CODE            INCIDENT NUMBER
524 2 05 27 1 001     15983              Kathmandu M.N.P.
524 2 05 27 1 001 11  15993              Kathmandu M.N.P.
Name: Municipality/VDC, dtype: object 

Duplicate Municipalit

Description Alert: Mahottari/Jun 15, Rajapa karyakarta dwara Budhabar bihana Jaleshwor sthit Sanghiya Samajbadi Forum Nepal ko karyalaya maa todfod.

In Mahottari, a group of RJP-N supporters vandalised the party office of Sanghiya Samajbadi Forum Nepal on Wednesday morning. The incident was condemned by Ram Naresh Yadav, the district chairman of Sanghiya Samajbadi Forum Nepal.

à¤°à¤¾à¤·à¥à¤Ÿà¥à¤°à¤¿à¤¯ à¤œà¤¨à¤¤à¤¾ à¤ªà¤¾à¤°à¥à¤Ÿà¥€ à¤•à¤¾à¤°à¥à¤¯à¤•à¤°à¥à¤¤à¤¾à¤²à¥‡ à¤®à¤¹à¥‹à¤¤à¥à¤¤à¤°à¥€ à¤¸à¤¦à¤°à¤®à¥à¤•à¤¾à¤°à¤® à¤œà¤²à¥‡à¤¶à¥à¤µà¤°à¤¸à¥à¤¥à¤¿à¤¤ à¤¸à¤‚à¤˜à¥€à¤¯ à¤¸à¤®à¤¾à¤œà¤µà¤¾à¤¦à¥€ à¤«à¥‹à¤°à¤® à¤¨à¥‡à¤ªà¤¾à¤²à¤•à¥‹ à¤•à¤¾à¤°à¥à¤¯à¤¾à¤²à¤¯à¤®à¤¾ à¤†à¤•à¥à¤°à¤®à¤£ à¤—à¤°à¥‡à¤•à¤¾ à¤›à¤¨à¥ à¥¤
 
à¤•à¤°à¤¿à¤¬ à¤¡à¥‡à¤¢ à¤¦à¤°à¥à¤œà¤¨ à¤¸à¤‚à¤–à¥à¤¯à¤¾à¤®à¤¾ à¤°à¤¾à¤œà¤ªà¤¾ à¤•à¤¾à¤°à¥à¤¯à¤•à¤°à¥à¤¤à¤¾ à¤¬à¥à¤§à¤¬à¤¾à¤° à¤¬à¤¿à¤¹à¤¾à¤¨ à¤«à¥‹à¤°à¤® à¤•à¤¾à¤°à¥à¤¯à¤¾à¤²à¤¯ à¤ªà¥à¤—à¥‡à¤° à¤à¤¨à¥à¤¡à¤¾ à¤‰à¤–à¥‡à¤²à¥‡à¤° à¤«à¥à¤¯à¤¾à¤à¤•à¥à¤¨à¥à¤•à¤¾ à¤¸à

lat/lon: 26.541946 86.756722
HLCIT CODE: 524 1 03 15 3 001
District                       Saptari
Municipality/VDC         Rajbiraj N.P.
Ward                               NaN
Municipality/VDC(New)         Rajbiraj
Ward(New)                          NaN
Name: 15010, dtype: object 

Description Alert: Saptari/Jun 12, Sambidhan samsodhan ko maag gardai Rajapa dwara Sombar Rajbiraj ma mashal julush pradarshan.

Cadres of the agitating Rastriya Janata Party Nepal (RJPN) organised a torch rally at Rajbiraj, the district headquarters of Saptari, on Monday.

The rally that started from Shiva Mandir marched through varoius parts of Rajbiraj and converged into a corner assembly at Janaandolan Chok.

RJPN central senior vice-chairperson Mrigendra Singh Yadav and Nawal Kishor Sah and General Secretary Manish Kumar Suman, among other leaders, took part in the rally.

RJPN on Sunday had announced fresh protest programmes including general strike saying that they will not take part in the upcoming s

lat/lon: 28.324825 82.658155
HLCIT CODE: 524 4 10 54 5 014 9
District                   Rolpa
Municipality/VDC         Dhawang
Ward                           9
Municipality/VDC(New)      Rolpa
Ward(New)                     10
Name: 15293, dtype: object 

Description Alert: Rolpa/Jun 28, Agyat samuha le matpetika jalaayepachhi Dhawang ko Janata Adharbhut bidhyalaya ko matdan sthagit.

An unidentified group has torched ballot box at polling center in Janata Adharbhut Bidhyalaya, Dhawang, of Rolpa district.

RSS reported some 15 ballot papers were burnt in the arson.

Some 150 votes were cast in the center. Voting has stopped in the polling center after the incident.

à¤œà¤¨à¤¤à¤¾ à¤†à¤§à¤¾à¤°à¤­à¥‚à¤¤ à¤µà¤¿à¤¦à¥à¤¯à¤¾à¤²à¤¯ à¤¥à¤µà¤¾à¤™ à¤°à¥‹à¤²à¥à¤ªà¤¾à¤®à¤¾ à¤®à¤¤à¤¦à¤¾à¤¨à¤•à¥‹ à¤•à¥à¤°à¤®à¤®à¤¾ à¤…à¤œà¥à¤žà¤¾à¤¤ à¤¸à¤®à¥‚à¤¹à¤²à¥‡ à¤®à¤¤à¤ªà¥‡à¤Ÿà¤¿à¤•à¤¾ à¤œà¤²à¤¾à¤à¤•à¤¾ à¤›à¤¨à¥ à¥¤ 

à¤°à¥‹à¤²à¥à¤ªà¤¾ à¤¨à¤—à¤°à¤ªà¤¾à¤²à¤¿à¤•à¤¾â€“à¥§à¥¦ à¤®à¤¾ à¤°à¤¹à¥‡à¤•à¥‹ à¤¸à¥‹ à¤®

Description Alert: Saptari/Jun 7, Hatya bhayeko bhandai mritak ka aafanta dwara Kanchanrup-9 sthit purba-paschim rajmarga ma chakkajam. 

East-west highway near Rupnagar of Kanchanpur municipality  in Saptari has been obstructed by the parents and relatives of Feku Yadav who was found hanged in a mango tree on 6th June near his house. Family of Feku Yadav is claiming that his death was a murder and not suicide. The locals and relative of the deceased obstructed the road in the morning of 7th June from 8 to 9am. The road obstruction ended after a police team of Kanchanpur area police office came and gave commitment to investigate and punish the offender. According to the deceased Yadav's mother he had dispute with some locals on Monday regarding a land issue.

à¤¸à¤ªà¥à¤¤à¤°à¥€à¤•à¥‹ à¤•à¤‚à¤šà¤¨à¤°à¥à¤ª à¤¨à¤—à¤°à¤ªà¤¾à¤²à¤¿à¤•à¤¾ à¥¯ à¤°à¥à¤ªà¤¨à¤—à¤°à¤¸à¥à¤¥à¤¿à¤¤ à¤ªà¥‚à¤°à¥à¤µ à¤ªà¤¶à¥à¤šà¤¿à¤® à¤°à¤¾à¤œà¤®à¤¾à¤°à¥à¤—à¤®à¤¾ à¤šà¤•à¥à¤•à¤¾à¤œà¤¾à¤® à¤­à¤¯à¥‹ à¥¤

à¤¸à¤¡à¤•à¤

lat/lon: 27.709271 85.315237
HLCIT CODE: 524 2 05 27 1 001 1
District                        Kathmandu
Municipality/VDC         Kathmandu M.N.P.
Ward                                    1
Municipality/VDC(New)           Kathmandu
Ward(New)                               1
Name: 13827, dtype: object 

Description Alert: Kathmandu/Mar 20, Nirwachan ayog gherau garna gayeka RaPraPa kaaryakarta ra prahari bich jhadap huda darzan badhi ghaite.

The Rastriya Prajatantra Party on Monday picketed the Election Commission in Kathmandu, protesting the constitutional bodyâ€™s decision to delete a section of the partyâ€™s charter about the Hindu state and the monarchy.

The party Chairman and Deputy Prime Minister and Minister for Federal Affairs and Local Development Kamal Thapa himself led the protest.

Meanwhile, a clash erupted between the RPP cadres and police outside the EC. Police have lobbed tear gas shells and baton-charged to disperse the mob.

It has been learned that dozens of RPP cadres,

lat/lon: 26.6882 87.263291
HLCIT CODE: 524 1 02 06 5 007
District                    Sunsari
Municipality/VDC         Hanshposha
Ward                            NaN
Municipality/VDC(New)       Itahari
Ward(New)                       NaN
Name: 17436, dtype: object 

Description Alert: Sunsari/Dec 22, Rajmarga ma City Safari gudauna pratibandha lagayeko birodh ma utriyeka chalak ra prahari bich jhadap. 

A clash erupted between e-rickshaw operators and police in Itahari Municipality. The clash took place when e-rickshaw operators staged protest against a new rule of banning three-wheelers.

à¤°à¤¾à¤œà¤®à¤¾à¤°à¥à¤—à¤®à¤¾ à¤—à¥à¤¡à¤¾à¤‰à¤¨ à¤ªà¥à¤°à¤¤à¤¿à¤¬à¤¨à¥à¤§ à¤²à¤—à¤¾à¤à¤•à¥‹ à¤µà¤¿à¤°à¥‹à¤§à¤®à¤¾ à¤‰à¤¤à¥à¤°à¤¿à¤à¤•à¤¾ à¤µà¤¿à¤¦à¥à¤¯à¥à¤¤à¥€à¤¯ à¤°à¤¿à¤•à¥à¤¸à¤¾ (à¤¸à¤¿à¤Ÿà¥€ à¤¸à¤«à¤¾à¤°à¥€) à¤šà¤¾à¤²à¤• à¤° à¤ªà¥à¤°à¤¹à¤°à¥€à¤¬à¥€à¤š à¤à¤¡à¤ª à¤­à¤à¤•à¥‹ à¤›à¥¤ à¤à¤¡à¤ªà¤®à¤¾ à¥¬ à¤°à¤¾à¤‰à¤¨à¥à¤¡ à¤… à¤¶à¥à¤°à¥à¤—à¥à¤¯à¤¾à¤¸ à¤ªà¥à¤°à¤¹à¤¾à¤° à¤—à¤°à¥‡à¤•à¥‹ à

## full17df: Total Duplicate Municipality/VDC found: 403
## ND17df: Total Duplicate Municipality/VDC found: 344 

There are about 60 duplicates that have been dropped in the ND17df file that are in the same municipality VDC. It is therefore more effective to find duplicates by location than by title. 